In [1]:
!pip install PyPDF2 langchain==0.3.9 langchain-community==0.3.9 langchain-core==0.3.21 langchain-experimental==0.3.3 huggingface-hub==0.23.2 instructor==1.7.0 InstructorEmbedding==1.0.1 sentence-transformers==2.2.2 torch pdfplumber faiss-cpu

In [2]:
!pip install langchain_ollama==0.2.0

In [3]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_experimental.text_splitter import SemanticChunker
import os
from langchain.embeddings import OllamaEmbeddings
import pdfplumber
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from InstructorEmbedding import INSTRUCTOR
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama
from langchain_ollama.llms import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

c:\Users\L03564299\AppData\Local\miniconda3\Lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


The following function loads the pdf file and returns the vectorstore. Do not modify. You can run it with the command:

load_db(embeddings,pdf_path)

where "embeddings" and "pdf_path" have been previously defined.

In [4]:
## Do not mofify
def load_db(embeddings,pdf_path):
    text =''
    with open(pdf_path,'rb') as file:
        pdf_reader = PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text()

    text_splitter=SemanticChunker(embeddings, breakpoint_threshold_type="percentile")
    #text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_text(text)
    # define embedding
    vectorstore = FAISS.from_texts(docs, embeddings)
    return vectorstore

Define the embeddings. You can choose the embeddings you prefer, here we have the option of Instructor Embeddings from huggingface (not local) or Ollama embeddings (local).

In [5]:
#embeddings = HuggingFaceInstructEmbeddings(model_name='hkunlp/instructor-xl')
from langchain.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(
    model="llama3.2",
)

C:\Users\L03564299\AppData\Local\Temp\ipykernel_36052\943042438.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(


Define the name and path of the pdf file to load. Modify at your needs:

In [6]:
pdf_path = "PeterPan.pdf"

In the following piece of code we check whether the file with the embeddings already exists (stored in a directory called "faiss_index"). If the directory doesn't exists the embeddings are created calling the function "load_db" previously defined. The embeddings for the same pdf file can be created only once.
Do not modify.

In [7]:
#Do not modify 
import os
if not os.path.exists('faiss_index'):
    print('faiss_index doesnt exist')
    vectorstore=load_db(embeddings,pdf_path)
    vectorstore.save_local("faiss_index")
else:
    print('loading faiss_index')
    vectorstore = FAISS.load_local("faiss_index",embeddings=embeddings,allow_dangerous_deserialization=True)


loading faiss_index


Create the document retriever:

In [8]:
#Do not modify
retriever = vectorstore.as_retriever()

Personalize your prompt template:

In [9]:
template = """Eres un asistente para contestar preguntas del ususario sobre el contenido del file subido, que trata de la historia de Peter Pan. 
    Contesta siempre en español y agradece el usuario por preguntar. Si las preguntas son sobre otro tema, contesta que no puedes contestar.
    {context}
    Question: {question}
    Helpful Answer:"""
qa_prompt = ChatPromptTemplate.from_template(template)

Create the comnversation chain. The chain is a "ConversionalRetrievalChain" to take information only from the retriever (the pdf file). The llm is used only to interpret the questions and formulate the text of the answers. The chat memory is already included in the chain.


In [10]:
#Do not modify
llm = OllamaLLM(model='llama3.2')
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": qa_prompt}
)

C:\Users\L03564299\AppData\Local\Temp\ipykernel_36052\2042350049.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [11]:
history = []

Loop to handle the chat:

In [13]:
def handle_conversation(conversation,history):
    print('Escribe "exit" para salir')
    while True:
        question = input('Tu: ')
        if question.lower() == 'exit':
            break
        print(question)
        result = conversation({"question": question}, {"chat_history": history})
        history.append((question, result["answer"]))
        print("Bot: ",result['answer'])

handle_conversation(conversation_chain,history)

Escribe "exit" para salir


C:\Users\L03564299\AppData\Local\Temp\ipykernel_36052\387802828.py:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = conversation({"question": question}, {"chat_history": history})
